<a href="https://colab.research.google.com/github/Ashish2Parimi/Telugu_Headline_Generator/blob/main/word2vec_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import io
import re
import string
import tensorflow as tf
import tqdm
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [7]:
data = pd.read_csv('/content/data.csv')
data = data.applymap(lambda x: str(x) if isinstance(x, int) or isinstance(x, float) else x)

d = list(data['heading']+' '+data['body'])
d = ' '.join(d)
text_file = open("data.txt", "w")
n = text_file.write(d)
text_file.close()

In [8]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [9]:
path_to_file = '/content/data.txt'

In [10]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds

<FilterDataset shapes: (), types: tf.string>

In [11]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [12]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re
import string
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [13]:
vectorize_layer.adapt(text_ds.batch(1024))

In [14]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()


In [15]:
vocab = vectorize_layer.get_vocabulary()


In [16]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [17]:
write_vocab_file('vocab.txt', vocab)


In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

3886


In [ ]:
# for seq in sequences[:5]:
#   print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

NameError: ignored

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)
print(len(targets), len(contexts), len(labels))

100%|██████████| 3886/3886 [00:07<00:00, 513.76it/s]

16197 16197 16197


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3, 2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = self.dots([context_emb, word_emb])
    return self.flatten(dots)

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 128
num_ns = 4
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")


In [ ]:
word2vec.fit(dataset, epochs=50, callbacks=[tensorboard_callback])


Epoch 1/50
15/15 [==============================] - 4s 45ms/step - loss: 1.6087 - accuracy: 0.2424
Epoch 2/50
15/15 [==============================] - 0s 11ms/step - loss: 1.5909 - accuracy: 0.7616
Epoch 3/50
15/15 [==============================] - 0s 12ms/step - loss: 1.5624 - accuracy: 0.8816
Epoch 4/50
15/15 [==============================] - 0s 11ms/step - loss: 1.5094 - accuracy: 0.8798
Epoch 5/50
15/15 [==============================] - 0s 11ms/step - loss: 1.4234 - accuracy: 0.8571
Epoch 6/50
15/15 [==============================] - 0s 12ms/step - loss: 1.3067 - accuracy: 0.8350
Epoch 7/50
15/15 [==============================] - 0s 11ms/step - loss: 1.1755 - accuracy: 0.8156
Epoch 8/50
15/15 [==============================] - 0s 11ms/step - loss: 1.0525 - accuracy: 0.8025
Epoch 9/50
15/15 [==============================] - 0s 12ms/step - loss: 0.9531 - accuracy: 0.7942
Epoch 10/50
15/15 [==============================] - 0s 12ms/step - loss: 0.8791 - accuracy: 0.7912
Epoch 11/

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
# %reload_ext tensorboard

# %tensorboard --logdir logs


In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
word_vec = {}

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
  word_vec[word] = vec
out_v.close()
out_m.close()

In [ ]:
# word2vec.save("word2vec")

In [ ]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
#   files.download('/content/word2vec')
# except Exception:
#   pass

In [ ]:
word_vec 

In [ ]:
import csv
with open('word_vec.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in word_vec.items():
       writer.writerow([key, value])